# Trainer

> Fill in a module description here

In [ ]:
#| default_exp trainer

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import torch
from torch import optim
from gpt2.model import GPT2

In [ ]:
#| export
d_model = 512
d_ff = 2048

batch_size = 2
context_length = 256

max_iters = 2
eval_interval = 500
eval_iters = 200
learning_rate = 3e-4

n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#| export
with open('../data/shakespeare.txt', 'r', encoding='utf8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

/var/folders/pf/g3nr86yn4j71vzzmv6knwdhr0000gp/T/ipykernel_40225/4015623181.py:15: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:77.)
  data = torch.tensor(encode(text), dtype=torch.long)


In [ ]:
#| export
model = GPT2(
    n_head=n_head, n_layer=3, vocab_size=vocab_size,
    n_embd=n_embd, context_length=context_length,
).to(device)

In [ ]:
#| export
sum(p.numel() for p in model.parameters())

5468993

### Utils

In [ ]:
#| export
# data loading
def get_batch(split, context_length: int):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_length, (batch_size,))
    x = torch.stack([data[i:i+context_length] for i in ix])
    y = torch.stack([data[i+1:i+context_length+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
#| export
@torch.no_grad()
def estimate_loss(model, eval_iters: int, context_length: int):
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, context_length)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        
        out[split] = losses.mean()
    
    model.train()
    return out

### Training Loop

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
for iter in range(max_iters):
    
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model, eval_iters, context_length)
        print(f"step {iter}, train loss = {losses['train']:.3f}, val loss = {losses['val']:.3f}")
    
    xb, yb = get_batch('train', context_length)
    
    # evaluate the loss
    logits, loss = model(xb, yb)
    print(f"loss={loss}")
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

step 0, train loss = nan, val loss = nan
loss=nan
step 1, train loss = nan, val loss = nan
loss=nan
